In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import datetime as dt
import locale
import os

### Comuni BDN

In [2]:
gdf_comuni = gpd.read_file(r"../input/shp/centroidi_comuni_bdn.shp")[['ISTAT','geometry']]
gdf_comuni.rename(columns={'ISTAT':'CodIstat', 'COMUNE':'Comune'}, inplace=True)
gdf_comuni["CodIstat"] = gdf_comuni["CodIstat"].astype(int)
gdf_comuni.head(5)

,CodIstat,geometry
0,78019,POINT (15.91568 39.57404)
1,90015,POINT (8.74288 40.52530)
2,22013,POINT (11.14220 45.95304)
3,33003,POINT (9.97993 44.97384)
4,78024,POINT (16.51171 40.09050)


### Dati USUTU

Raggruppamento dati usutu con conteggio numero di insetti e uccelli

In [4]:
df_usutu = pd.read_excel(r"../input/xls/usutu.xlsx")

df_insetti = df_usutu.query("SpecieCampione == 'CULEX PIPIENS'")
df_uccelli = df_usutu.query("SpecieCampione == 'UCCELLO SELVATICO'")

df_insetti_group = df_insetti.groupby(['Anno','Sede','Provincia','Comune'])["Comune"].count().reset_index(name="N_insetti")
df_uccelli_group = df_uccelli.groupby(['Anno','Sede','Provincia','Comune'])["Comune"].count().reset_index(name="N_uccelli")

df_usutu_group = df_insetti_group.append(df_uccelli_group).fillna(0)
df_usutu_group['N_insetti'] = df_usutu_group['N_insetti'].astype(int)
df_usutu_group['N_uccelli'] = df_usutu_group['N_uccelli'].astype(int)

df = df_usutu_group.groupby(['Anno','Sede','Provincia','Comune'],as_index=False).agg('sum')

df.query('N_insetti > 0 & N_uccelli > 0')


,Anno,Sede,Provincia,Comune,N_insetti,N_uccelli
5,2018.0,TE,ASCOLI PICENO,MONTEPRANDONE,2,2
6,2018.0,TE,ASCOLI PICENO,SAN BENEDETTO DEL TRONTO,1,2
9,2018.0,TE,BOLOGNA,BOLOGNA,11,104
10,2018.0,TE,BOLOGNA,BUDRIO,4,1
18,2018.0,TE,BOLOGNA,SAN GIOVANNI IN PERSICETO,19,3
27,2018.0,TE,FERRARA,ARGENTA,4,4
28,2018.0,TE,FERRARA,BONDENO,4,3
29,2018.0,TE,FERRARA,COMACCHIO,6,1
31,2018.0,TE,FERRARA,FERRARA,16,21
35,2018.0,TE,FERRARA,OSTELLATO,4,2


### Esportazione